# 유투브 채널 수리키친 더보기란 웹 스크랩핑

In [2]:
import pandas as pd
import threading
import re
import time
import random
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.action_chains import ActionChains

In [ ]:
# selenimu으로 크롬드라이버 실행
driver = webdriver.Chrome("c:/my_web/chromedriver/chromedriver.exe")

In [ ]:
# 페이지 scroll을 내리는 함수 
def scroll() :
    try :
        # 페이지 내 스크롤 높이 받아오기
        last_page_height = driver.execute_script("return document.documentElement.scrollHeight")
        while True : # true일 동안만 돌리기 
            # 임의의 페이지 로딩시간 정하기 ## pc환경에 따라 로딩시간 최적화를 통해 로딩시간 단축
            pause_time = random.uniform(1,2) # 인자로 들어온 a~b 사이의 실수(float)를 반환
            # 페이지 최하단까지 스크롤
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight)")
            time.sleep(pause_time) # 일정 시간동안 프로세스를 일시정지. ##실수단위로도 지정할 수 있어 정교한 시간 제어가 가능 
            # 무한 스크롤 동작을 위해 살짝 위로 스크롤(i.e., 페이지를 위로 올렸다가 내리는 제스쳐)
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight)")
            # 페이지 내 스크롤 높이 새로 받아오기
            new_page_height = driver.execute_script("return document.documentElement.scrollHeight")
            # 스크롤을 완료한 경우(더이상 페이지 높이 변화가 없는 경우)
            if new_page_height == last_page_height:
                print("스크롤 완료")
                break
            # 스크롤 완료하지 않은 경우, 최하단까지 스크롤
            else:
                last_page_height = new_page_height
                
    except Exception as e:
        print("에러 발생: ", e)

## 1. 각 동영상별 링크, 제목 스크랩

In [ ]:
# 유투브 수리키친 채널 페이지 페이지 response
driver.get("https://www.youtube.com/@suri3180/videos")
time.sleep(3)
scroll()
html = driver.page_source
soup = bs(html, 'html.parser')

In [ ]:
# scrap
# box 
boxes = soup.select("ytd-rich-item-renderer")
print(len(boxes))

box = boxes[0] # 292개 중 첫 번째 동영상에 해당되는 태그
print(len(box))
print(box)

In [ ]:
#### 모든 'a' 요소의 href 뽑아오기 ###
links_titles = []
for box in boxes :
    for a in box.select('a') : # 첫 box에 해당하는 모든 'a'요소
        title = a.text
        if a.has_attr('href'): # has_attr : href의 존재여부 확인
            link = 'https://www.youtube.com/' + a['href']
    links_titles.append([link,title])

# 스크랩한 데이터 df로 변환
data = {'link' : [link for link, title in links_titles], 'title' : [title for link, title in links_titles]}
df = pd.DataFrame(data)
df.head()

In [ ]:
# csv 파일로 저장
df.to_csv('c:/my_web/multi_project/semi/link_title.csv', encoding = 'utf-8')

## 2. 각 링크별 재료 스크랩

In [ ]:
final_ingres = []
final_link = []
for link, title in links_titles :
    driver.get(link)
    time.sleep(3)
    scroll()
    try :
        # 더보기 버튼 나타날 때 까지 기다리기
        more_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '/html/body/ytd-app/div[1]/ytd-page-manager\
                                            /ytd-watch-flexy/div[5]/div[1]/div/div[2]/ytd-watch-metadata/\
                                            div/div[3]/div[1]/div/ytd-text-inline-expander/tp-yt-paper-button[1]')))
        # ActionChains 클래스는 사용자가 마우스 또는 키보드로 수행하는 작업을 시뮬레이트하는 데 사용
        # move_to_element() 메소드는 주어진 요소로 마우스 커서를 이동시키는 데 사용
        # click() 메소드는 마우스 왼쪽 버튼을 클릭하는 데 사용
        # perform() 메소드는 이러한 동작을 실행
        # ActionChains(driver).move_to_element(more_button).click().perform() 코드는 
            # driver 객체를 사용하여 more_button 요소로 마우스를 이동한 다음 해당 요소를 클릭
        ActionChains(driver).move_to_element(more_button).click().perform() # 버튼 클릭
        
        # 버튼 클릭 후 새로운 데이터가 뜰 때 까지 기다리기
        new_data = WebDriverWait(driver,10).until(
            EC.presence_of_element_located((By.XPATH, '/html/body/ytd-app/div[1]/ytd-page-manager/ytd-watch-flexy/div[5]/div[1]/div/div[2] \
                                            /ytd-watch-metadata/div/div[3]/div[1]/div/ytd-text-inline-expander/yt-attributed-string')))
    except :
        print('새로운 데이터가 로드되지 않았습니다.')

    html = driver.page_source
    soup = bs(html, 'html.parser')
    
    # 재료 text 스크랩 하기
    description = soup.select('div#description')[0]
    ingres = description.select('span')[3].text
    if ingres != '' :
        ingres = ingres.split('\n')
        ingres = list(filter(lambda x : x != '', ingres))
        final_ingres.append(ingres)
        final_link.append(link)

print(final_ingres)
print(len(final_ingres))        

In [ ]:
# list타입을 numpy 배열로 변환
ingres_arr = np.array(final_ingres)
link_arr = np.array(final_link)

# df로 변환
df_link = pd.DataFrame(link_arr)
df_ingres = pd.DataFrame(ingres_arr)

# link DF와 ingres DF 합치기(concat)
final_df = pd.concat([df_link, df_ingres], axis = 1)
final_df.columns = ['link', 'ingredient']

# csv 파일로 저장 
final_df.to_csv('c:/my_web/multi_project/semi/final_df.csv', encoding = 'utf-8')
final_df.head()

In [ ]:
# 저장한 두 df 불러오기
ingres_ = pd.read_csv('c:/my_web/multi_project/semi/final_df.csv', encoding = 'utf-8')
title_ = pd.read_csv('c:/my_web/multi_project/semi/link_title.csv', encoding = 'utf-8') 

# link를 기준으로 join해주기
df_right_join = pd.merge(title_, ingres_, left_on = 'link', right_on = 'link', how = 'right')
df_right_join.head() # 확인

# 불필요한 칼럼은 빼기 
really_final2 = df_right_join[['link', 'title', 'ingredient']]
really_final2.head() # 확인

# 저장
really_final2.to_csv('c:/my_web/multi_project/semi/really_final2.csv', encoding = 'utf-8')

# 확인 (index_col = 0 으로 불필요한 열은 제외시키기)
really_final2_test = pd.read_csv('c:/my_web/multi_project/semi/really_final2.csv', encoding = 'utf-8', index_col = 0)
really_final2_test.head() # 확인

## 3. Thumbnail 스크랩  

In [ ]:
import requests

for_thumbnail = pd.read_csv('c:/my_web/multi_project/semi/really_final3.csv', encoding = 'cp949', index_col = 0)
# for_thumbnail.head()


# 썸네일 뽑아오기
thumbnail_list = []
for link in df2['link'] :
    response = requests.get(link)
    html = response.text
    soup = bs(html, 'html.parser')
    # 썸네일 이미지 태그 찾아 해당 태그의 src 속성 찾아오기
    thumbnail = soup.find('link', {'itemprop' : 'thumbnailUrl'})
    thumbnail_url = thumbnail.get('href')
    thumbnail_list.append(thumbnail_url)
thumbnail_list

# df로 변환
image = pd.DataFrame(data = thumbnail_list)
image.columns = ['thumbnail']
# image.head()

# 기존 df와 병합(concat)
really_final4 = pd.concat([df2, image], axis=1)
# really_final4.head()

# 저장 
really_final4.to_csv('c:/my_web/multi_project/semi/really_final9.csv', encoding = 'utf-8')

In [ ]:
# 재료만 나오도록 전처리 
df_semi_fnal = pd.read_csv('c:/my_web/multi_project/semi/really_final9.csv', encoding = 'ansi')

df_semi_fnal["ingredient"] = df_semi_fnal["ingredient"].str.replace("[", "")
df_semi_fnal["ingredient"] = df_semi_fnal["ingredient"].str.replace("]", "")
df_semi_fnal['ingredient'] = df_semi_fnal['ingredient'].str.replace("'", '')
df_semi_fnal['ingredient'] = df_semi_fnal['ingredient'].replace(' ', '뭐먹지')
# df_semi_fnal.iloc[232]['ingredient'] ## 확인용

for i in range(len(df_semi_fnal)):
    if "재료," in df15.iloc[i]['ingredient']:
        if ", *" in df15.iloc[i]['ingredient']:
            df15.iloc[i]['ingredient'] = df15.iloc[i]['ingredient'].split("재료,")[1].split(", *")[0].strip()
        elif ", ?" in df15.iloc[i]['ingredient']:
            df15.iloc[i]['ingredient'] = df15.iloc[i]['ingredient'].split("재료,")[1].split(", ?")[0].strip()
        else:
            df15.iloc[i]['ingredient'] = df15.iloc[i]['ingredient'].split("재료,")[1].strip()
    else:
        continue
        
df_semi_fnal.to_csv('c:/my_web/multi_project/semi/really_final9.csv', encoding = 'utf-8')